# Import librarys and data


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from random import randrange, uniform

import shutil
import itertools
from time import time

#ignore warning messages 
import warnings
warnings.filterwarnings('ignore') 

sns.set()

output_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
                 'A','B','C','D','E','F','G','H','I','J','K','L','M',
                 'N', 'O','P','Q','R','S', 'T','U','V','W','X','Y','Z']

- Mnist dataset link : https://www.kaggle.com/oddrationale/mnist-in-csv
- Kaggle A-Z dataset link : https://www.kaggle.com/sachinpatel21/az-handwritten-alphabets-in-csv-format
- **NB** : I combined test and train for mnist because train and test will be on new dataset which is a combination of Kaggle A-Z and mnist.

In [ ]:
# Kaggle A-Z dataset
kaggle_az = pd.read_csv("dataset/a_z.csv", header=None)

In [ ]:
# MNIST DATASET
mnist = pd.read_csv("dataset/mnist.csv", header=None)

# Explore

In [ ]:
kaggle_az.head()

In [ ]:
mnist.head()

# Concate the two dataframes

- New labels : 
    - 0-9 : 0-9.
    - A-Z : 10-35. Means we will add 10 to all values of the first column (labels)


In [ ]:
kaggle_az[0] = kaggle_az[0] + 10

In [ ]:
kaggle_az

In [ ]:
# Combine the two dataframes:

X_kaggle = kaggle_az.drop(0,axis = 1)
y_kaggle = kaggle_az[0]

X_mnist = mnist.drop(0,axis = 1)
y_mnist = mnist[0]

In [ ]:
X_kaggle.shape[0]-X_mnist.shape[0]

In [ ]:
X = np.concatenate((X_kaggle, X_mnist))
y = np.concatenate((y_kaggle, y_mnist))

In [ ]:
# Shuffle the data, we do not shuffle X and y sepratly otherwise we will lose the true labels
data = []
data.append([X, y])
random.shuffle(data)

In [ ]:
# Retreive X and y from data :
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, 28, 28, 1)
X = X.astype(int)
y = np.array(y)
y = y.reshape((y.shape[1], 1)) # we do not want (1, n)

In [ ]:
print(X.shape, y.shape)

In [ ]:
# Save data in pickle format
pickle_out = open("dataset_pickle/X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("dataset_pickle/y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
# Load the data
pickle_in = open("dataset_pickle/X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("dataset_pickle/y.pickle","rb")
y = pickle.load(pickle_in)

- Let's plot random sample from the entire dataset:

In [ ]:
random_index = random.randrange(0, len(X))
sample = X[random_index]
sample = sample.astype(int)
plt.imshow(sample)
plt.title(output_labels[int(y[random_index])])
plt.show()

# Data preparation

In [ ]:
# splite the data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25)

# Normalize the data
X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)

In [ ]:
# One hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build the model


In [ ]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(output_labels), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.25, epochs=5, batch_size=20)

In [ ]:
scores = model.evaluate(X_test,y_test)

In [ ]:
random_index = random.randrange(0, len(X_test))
sample = X_test[random_index]
image = np.expand_dims(sample, axis=0)
pred = np.argmax(model.predict(image)[0])
plt.imshow(sample)
plt.title(f"Prediction: {output_labels[pred]}")
plt.show()